In [1]:
# script to add a new species to the species_list

# TODO - check that the informal name isn't already taken, add -n+1 to the end

In [2]:
import pandas as pd
import os
import sys
sys.path.append('/work/westgroup/harris.se/autoscience/autoscience_workflow/workflow/scripts/thermo/')
import job

In [3]:
def species_exists_in_db(species_smiles):
    try:
        job.smiles2index(species_smiles)
        return True
    except IndexError:
        # not found, which means we can add it
        return False

In [4]:
# load the base model
import os
import pickle
import datetime
import subprocess
import numpy as np

import rmgpy.chemkin
import rmgpy.data.kinetics
# import cantera as ct


import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# Load the base model and the full 1 week model
basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_model'
base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)
print(f'{len(species_list)} species, {len(reaction_list)} reactions')


110 species, 1822 reactions


In [6]:
# Load the base model and the full 1 week model
basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_1week'
base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list2, reaction_list2 = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)
print(f'{len(species_list2)} species, {len(reaction_list2)} reactions')


130 species, 2488 reactions


In [7]:
for i in range(0, len(species_list)):
    match = False
    for j in range(0, len(species_list2)):
        if species_list[i].is_isomorphic(species_list2[j]):
            match = True
            break
    if not match:
        print(species_list[i], 'not in mechanism 2')

In [8]:
species_to_add = []
for i in range(0, len(species_list2)):
    match = False
    for j in range(0, len(species_list)):
        if species_list2[i].is_isomorphic(species_list[j]):
            match = True
            break
    if not match:
        print(species_list2[i].smiles, '\tnot in mechanism 1')
        species_to_add.append(species_list2[i])

C#C[O] 	not in mechanism 1
[CH2]O 	not in mechanism 1
C=CO 	not in mechanism 1
[CH2]CO 	not in mechanism 1
C=C[C]=O 	not in mechanism 1
C=CCO 	not in mechanism 1
CC=CO 	not in mechanism 1
C[CH]C(C)O 	not in mechanism 1
CC(O)C(C)O[O] 	not in mechanism 1
[CH]C 	not in mechanism 1
[CH2]C[CH2] 	not in mechanism 1
O=O 	not in mechanism 1
[O]C[CH]O 	not in mechanism 1
OC1CO1 	not in mechanism 1
[CH2]C(O)C=O 	not in mechanism 1
C[C]CO 	not in mechanism 1
[O]OCC(O)C=O 	not in mechanism 1
O=[C]C(O)COO 	not in mechanism 1
O[CH]COO 	not in mechanism 1
C=CC(=O)O[O] 	not in mechanism 1


In [18]:
# Read the species csv
DFT_DIR = '/work/westgroup/harris.se/autoscience/autoscience/butane/dft'
species_csv = os.path.join(DFT_DIR, 'species_list.csv')
species_df = pd.read_csv(species_csv)

if 'Unnamed: 0' in species_df.keys():  
    species_df = species_df.drop(columns=['Unnamed: 0'])

print(species_df)

       i                 name        SMILES
0      0                   N2           N#N
1      1                   Ar          [Ar]
2      2                   He          [He]
3      3                   Ne          [Ne]
4      4            butane(1)          CCCC
..   ...                  ...           ...
125  125         C[C]CO(9122)        C[C]CO
126  126  [O]OCC(O)C=O(11705)  [O]OCC(O)C=O
127  127  O=[C]C(O)COO(11851)  O=[C]C(O)COO
128  128      O[CH]COO(12103)      O[CH]COO
129  129  C=CC(=O)O[O](13050)  C=CC(=O)O[O]

[130 rows x 3 columns]


In [ ]:
# add all the new species:
starting_length = len(species_df)
for i in range(0, len(species_to_add)):
    if species_to_add[i].smiles in species_df['SMILES'].values or species_exists_in_db(species_to_add[i].smiles):
        print(f'skipping {species_to_add[i]} because it was already added')
        continue
    new_entry = {
        'i': starting_length + i,
        'name': str(species_to_add[i]),
        'SMILES': species_to_add[i].smiles
    }
    print(f'adding new entry {new_entry}')
    species_df = species_df.append(new_entry, ignore_index=True)

In [ ]:
species_df

In [ ]:
# save the new results
species_df.to_csv(species_csv)

In [10]:
# add a specific species
import rmgpy.species
import rmgpy

In [11]:
family = 'Disproportionation'
spec_dict = rmgpy.chemkin.load_species_dictionary(os.path.join(rmgpy.settings['database.directory'], 'kinetics', 'families', family, 'training', 'dictionary.txt'))


In [13]:
new_species = spec_dict['C6H5']

In [15]:
# check if it's already in there
smiles = new_species.smiles
already_in_db = False
if smiles in species_df['SMILES'].values:
    print(f'skipping {new_species} because it was already added')
    already_in_db = True
if species_exists_in_db(smiles):
    print(f'skipping {new_species} because it was already added')
    already_in_db = True

In [16]:
# add single new species:
if not already_in_db:
    new_entry = {
        'i': len(species_df),
        'name': str(new_species),
        'SMILES': smiles
    }
    species_df = species_df.append(new_entry, ignore_index=True)


In [17]:
species_df

,i,name,SMILES
0,0,N2,N#N
1,1,Ar,[Ar]
2,2,He,[He]
3,3,Ne,[Ne]
4,4,butane(1),CCCC
...,...,...,...
126,126,[O]OCC(O)C=O(11705),[O]OCC(O)C=O
127,127,O=[C]C(O)COO(11851),O=[C]C(O)COO
128,128,O[CH]COO(12103),O[CH]COO
129,129,C=CC(=O)O[O](13050),C=CC(=O)O[O]


In [24]:
# add everything in the disproportionation family
import rmgpy.species
import rmgpy


family = 'Disproportionation'
spec_dict = rmgpy.chemkin.load_species_dictionary(os.path.join(rmgpy.settings['database.directory'], 'kinetics', 'families', family, 'training', 'dictionary.txt'))

for key in spec_dict:
    print(spec_dict[key].smiles)
    
    # check if it's already in there
    smiles = spec_dict[key].smiles
    already_in_db = False
    if smiles in species_df['SMILES'].values:
        print(f'skipping {spec_dict[key]} because it was already added')
        already_in_db = True
    if species_exists_in_db(smiles):
        print(f'skipping {spec_dict[key]} because it was already added')
        already_in_db = True

    # add single new species:
    if not already_in_db:
        new_entry = {
            'i': len(species_df),
            'name': str(spec_dict[key]),
            'SMILES': smiles
        }
        species_df = species_df.append(new_entry, ignore_index=True)
        print(f'adding {spec_dict[key]}')
        
        

[C]#C
skipping C2H because it was already added
skipping C2H because it was already added
[CH2]O
skipping CH3O because it was already added
skipping CH3O because it was already added
C#C
skipping C2H2 because it was already added
skipping C2H2 because it was already added
C=O
skipping CH2O because it was already added
skipping CH2O because it was already added
[CH]=C
skipping C2H3 because it was already added
skipping C2H3 because it was already added
[O][O]
skipping O2 because it was already added
skipping O2 because it was already added
C#C
skipping C2H2_1 because it was already added
skipping C2H2_1 because it was already added
[O]O
skipping HO2 because it was already added
skipping HO2 because it was already added
C[CH2]
skipping C2H5 because it was already added
skipping C2H5 because it was already added
C=C
skipping C2H4 because it was already added
skipping C2H4 because it was already added
[CH2]
skipping CH2 because it was already added
skipping CH2 because it was already added

skipping S(186) because it was already added
CCC(C)OO
skipping S(187) because it was already added
skipping S(187) because it was already added
[CH]1CCOOC1
skipping S(5425) because it was already added
skipping S(5425) because it was already added
C1=COOCC1
skipping S(5951) because it was already added
skipping S(5951) because it was already added


In [25]:
species_df

,i,name,SMILES
0,0,N2,N#N
1,1,Ar,[Ar]
2,2,He,[He]
3,3,Ne,[Ne]
4,4,butane(1),CCCC
...,...,...,...
174,174,CH2NO,[O-][N+]=C
175,175,CHNO,[O-][N+]#C
176,176,C6H5,[c]1ccccc1
177,177,C6H6,c1ccccc1


In [26]:
# save the new results
species_df.to_csv(species_csv)